Defined Process 

1) Read the data into the variable. 
2) Converting the data into tuples. (Key value Pair)
3) Extracting each column to find max and min values for each column.
4) Creating the tuple after normalizing the data.

In [ ]:

# Mid-Term Project on Customer Segmentation for the subject of Big Data Engineering

# Project Members:
 
# Group No : 3

# Girish Khule - 23PGAI0057
# Minal Sheth - 23PGAI0079
# Raj Rajeshwari Bajre - 23PGAI0093
# Rohan Mehta - 23PGAI0001
# Satyam Verma - 23PGAI0065

# Method Used: Map-Reduce
# Platform: DataBricks
# No of Clusters: 3
# No of Iterations: 40



In [ ]:
#Command to initiate Spark
spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
#Command to initate sparkContext and assign alias "sc"
sc = spark.sparkContext

In [ ]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
#To check the files in the 'jio2022' directory

###Directory Checking

In [ ]:

%fs ls /FileStore/tables/jio2022

path,name,size,modificationTime
dbfs:/FileStore/tables/jio2022/apache_access.log,apache_access.log,160971,1659329943000
dbfs:/FileStore/tables/jio2022/bestcaptains/,bestcaptains/,0,0
dbfs:/FileStore/tables/jio2022/captains_ODI-1.csv,captains_ODI-1.csv,4510,1658751397000
dbfs:/FileStore/tables/jio2022/captains_ODI.csv,captains_ODI.csv,4510,1658751368000
dbfs:/FileStore/tables/jio2022/captains_Test-1.csv,captains_Test-1.csv,3332,1658751397000
dbfs:/FileStore/tables/jio2022/captains_Test.csv,captains_Test.csv,3332,1658751369000
dbfs:/FileStore/tables/jio2022/customerdata.csv,customerdata.csv,4499,1659180496000
dbfs:/FileStore/tables/jio2022/fooddelivery,fooddelivery,12346,1658571213000
dbfs:/FileStore/tables/jio2022/fooddelivery.txt,fooddelivery.txt,12346,1658571013000
dbfs:/FileStore/tables/jio2022/groceries.csv,groceries.csv,500843,1658836265000


###Section 1 - Demonstrate all steps and their results

###Reading The File Section

In [ ]:
#Reading data from CSV file.
def read_file ():
    consumer_data = sc.textFile("/FileStore/tables/jio2022/customerdata.csv")
    return ( consumer_data )

In [ ]:
#Reading Customer Data file
cus_data = read_file ()

In [ ]:
#First 10 values of the CSV file.
cus_data.take(10)

Out[8]: ['1,41100,48.75',
 '2,54100,28.1',
 '3,47800,46.75',
 '4,19100,40.25',
 '5,18200,35.8',
 '6,19800,41.45',
 '7,51500,31.55',
 '8,39900,48.2',
 '9,13000,36.8',
 '10,31900,43.05']

###Converting the text data to key, value tuple pair

In [ ]:
#Function to convert string values to a list of tuples splitted by commas, and convert salary and age values to float to maintain precision.
def convert_to_key_value_pair (line):
    
    tokens = line.split(",")
    consumer_data_rdd_salaries = float(tokens[1])
    consumer_data_rdd_age      = float(tokens[2])
    
    return ( tokens[0], (consumer_data_rdd_salaries, consumer_data_rdd_age) )

###Mapping Customer Data

In [ ]:
#Mapping the "convertToKeyValuePair" to each tuple of cus_data.
cus_data_tuple = cus_data.map( lambda x: convert_to_key_value_pair (x) )

In [ ]:
#print first 10 rows
cus_data_tuple.take(10)

Out[11]: [('1', (41100.0, 48.75)),
 ('2', (54100.0, 28.1)),
 ('3', (47800.0, 46.75)),
 ('4', (19100.0, 40.25)),
 ('5', (18200.0, 35.8)),
 ('6', (19800.0, 41.45)),
 ('7', (51500.0, 31.55)),
 ('8', (39900.0, 48.2)),
 ('9', (13000.0, 36.8)),
 ('10', (31900.0, 43.05))]

###Normalization of data

In [ ]:
#Calculating minimum/maximum salary and age by sorting through the list by values in ascending and descending order.Thereafter, taking 1st row elements to make MinMax dictionary.

def get_min_max_values ( customerDataRDD ):
    
    max_value_age     =  customerDataRDD.sortBy( lambda x: x[1][1], ascending=False).first()
    min_value_age     =  customerDataRDD.sortBy( lambda x: x[1][1], ascending=True).first()
    max_value_salary  =  customerDataRDD.sortBy( lambda x: x[1][0], ascending=False).first()
    min_value_salary  =  customerDataRDD.sortBy( lambda x: x[1][0], ascending=True).first()
    
    return ( {
                'max_value_age' : max_value_age[1][1], 
                'min_value_age' : min_value_age[1][1], 
                'max_value_salary' : max_value_salary[1][0], 
                'min_value_salary' : min_value_salary[1][0] 
             } )

In [ ]:
minMax = get_min_max_values( cus_data_tuple )

In [ ]:
#Printing Min Max values for Salary and Age from Data.
minMax

Out[14]: {'max_value_age': 50.4,
 'min_value_age': 26.7,
 'max_value_salary': 60200.0,
 'min_value_salary': 3100.0}

In [ ]:
#Normalization of salary and age between 0 to 1, as both need to be brought to the same scale in calculating the euclidian distance ahead.

def normalize_value( tuple_pair, minMaxDict ):
    
    tuplePairSalary = ( float(tuple_pair [1][0]) - minMaxDict['min_value_salary'] ) / ( minMaxDict['max_value_salary'] - minMaxDict['min_value_salary'] )
    tuplePairAge    = ( float(tuple_pair [1][1]) - minMaxDict['min_value_age'] ) / ( minMaxDict['max_value_age'] - minMaxDict['min_value_age'] )
    
    return ( tuple_pair[0], ( tuplePairSalary, tuplePairAge ) )

In [ ]:
#Mapping the "normalizeValue" to each tuple of cus_data_tuple.
cus_data_tuple_nor = cus_data_tuple.map( lambda x : normalize_value (x, minMax) )

In [ ]:
#Printing the 20 Normalize values.
cus_data_tuple_nor.take(20)

Out[17]: [('1', (0.6654991243432574, 0.9303797468354431)),
 ('2', (0.8931698774080561, 0.059071729957806)),
 ('3', (0.7828371278458844, 0.8459915611814347)),
 ('4', (0.28021015761821366, 0.5717299578059072)),
 ('5', (0.26444833625218916, 0.3839662447257383)),
 ('6', (0.29246935201401053, 0.6223628691983124)),
 ('7', (0.8476357267950964, 0.20464135021097052)),
 ('8', (0.6444833625218914, 0.9071729957805909)),
 ('9', (0.1733800350262697, 0.4261603375527425)),
 ('10', (0.5043782837127846, 0.6898734177215189)),
 ('11', (0.851138353765324, 0.12658227848101267)),
 ('12', (0.9387040280210157, 0.10759493670886079)),
 ('13', (0.9754816112084063, 0.12447257383966243)),
 ('14', (0.7600700525394045, 0.8649789029535867)),
 ('15', (0.9089316987740805, 0.06962025316455706)),
 ('16', (0.26444833625218916, 0.609704641350211)),
 ('17', (0.7583187390542907, 0.7784810126582279)),
 ('18', (0.7565674255691769, 0.7426160337552742)),
 ('19', (0.3327495621716287, 0.8417721518987342)),
 ('20', (0.87740805604203

In [ ]:
#Converting the normalized tuple to list of tuples.
cus_data_tuple_nor_list = cus_data_tuple_nor.collect()

In [ ]:
cus_data_tuple_nor_list

Out[38]: [('1', (0.6654991243432574, 0.9303797468354431)),
 ('2', (0.8931698774080561, 0.059071729957806)),
 ('3', (0.7828371278458844, 0.8459915611814347)),
 ('4', (0.28021015761821366, 0.5717299578059072)),
 ('5', (0.26444833625218916, 0.3839662447257383)),
 ('6', (0.29246935201401053, 0.6223628691983124)),
 ('7', (0.8476357267950964, 0.20464135021097052)),
 ('8', (0.6444833625218914, 0.9071729957805909)),
 ('9', (0.1733800350262697, 0.4261603375527425)),
 ('10', (0.5043782837127846, 0.6898734177215189)),
 ('11', (0.851138353765324, 0.12658227848101267)),
 ('12', (0.9387040280210157, 0.10759493670886079)),
 ('13', (0.9754816112084063, 0.12447257383966243)),
 ('14', (0.7600700525394045, 0.8649789029535867)),
 ('15', (0.9089316987740805, 0.06962025316455706)),
 ('16', (0.26444833625218916, 0.609704641350211)),
 ('17', (0.7583187390542907, 0.7784810126582279)),
 ('18', (0.7565674255691769, 0.7426160337552742)),
 ('19', (0.3327495621716287, 0.8417721518987342)),
 ('20', (0.87740805604203

###Setting up the initial Cluster points.

In [ ]:
#This is the function responsible for setting up the inital dictionary for cluster points that is being selected randomly from the customer data.
def set_initial_clusters_points ( consumer_data_rdd_nr_list ) :

    import random

    random.seed(0)  # remove this line, if you want different results for each run

    initial_sample_points = random.sample( consumer_data_rdd_nr_list, 3 )

    #initial_sample_points

    Clusters= {
                "Cluster_1"  :  ( initial_sample_points[0][1][0], initial_sample_points[0][1][1] ),
                "Cluster_2"  :  ( initial_sample_points[1][1][0], initial_sample_points[1][1][1] ),
                "Cluster_3"  :  ( initial_sample_points[2][1][0], initial_sample_points[2][1][1] ) 
               }
    
    return ( Clusters )

In [ ]:
#Setting Initial Cluster Points
cps = set_initial_clusters_points ( cus_data_tuple_nor_list )

In [ ]:
#Printing initial cluster points stored in Dict.
cps

Out[21]: {'Cluster_1': (0.2329246935201401, 0.5126582278481013),
 'Cluster_2': (0.87215411558669, 0.18776371308016876),
 'Cluster_3': (0.2381786339754816, 0.5696202531645571)}

In [ ]:
#The cluster values are being broadcasted.
def broad_cast_values ( clusterPoints ):
    
    ClustersVar = sc.broadcast( clusterPoints )
    
    return ( ClustersVar )

In [ ]:
cpsbc = broad_cast_values ( cps )

###Doing the First Iteration to assign clusters to all data points.

In [ ]:
#Function to calculate Euclidean Distance for all the data points with the 3 assumed points and consequntly assigning cluster to the points. 
def create_cluster ( tuple_pair, ClustersVar ):
    
    dist  = dict()
    dist_tuple = list()
    
    for cluster in ClustersVar.value.keys() :

        point_1 =  ( float(tuple_pair[1][0]), float(tuple_pair[1][1]) ) 
        point_2 =  (ClustersVar.value[cluster][0], ClustersVar.value[cluster][1])
        dist [cluster] = pow ( pow( point_1[0] - point_2[0], 2 ) + pow( point_1[1] - point_2[1], 2 ),  0.5 )
        

    if ( ( dist["Cluster_1"] < dist["Cluster_2"] ) and (dist["Cluster_1"] < dist["Cluster_3"] ) ):
        closet_cluster = "Cluster_1"
    elif ( ( dist["Cluster_2"] < dist["Cluster_1"] ) and (dist["Cluster_2"] < dist["Cluster_3"] ) ):
        closet_cluster = "Cluster_2"
    else:
        closet_cluster = "Cluster_3"
        
    return (closet_cluster, ( tuple_pair[0], tuple_pair[1][0], tuple_pair[1][1] ))
    
    

In [ ]:
#Mapping the 'createCluster' function to each value of the normalized customer tuple.
cus_data_clus = cus_data_tuple_nor.map(lambda x: create_cluster ( x, cpsbc ))

In [ ]:
#Printing the 10 values for the clusterised tuple data.
cus_data_clus.take(10)

Out[26]: [('Cluster_3', ('1', 0.6654991243432574, 0.9303797468354431)),
 ('Cluster_2', ('2', 0.8931698774080561, 0.059071729957806)),
 ('Cluster_3', ('3', 0.7828371278458844, 0.8459915611814347)),
 ('Cluster_3', ('4', 0.28021015761821366, 0.5717299578059072)),
 ('Cluster_1', ('5', 0.26444833625218916, 0.3839662447257383)),
 ('Cluster_3', ('6', 0.29246935201401053, 0.6223628691983124)),
 ('Cluster_2', ('7', 0.8476357267950964, 0.20464135021097052)),
 ('Cluster_3', ('8', 0.6444833625218914, 0.9071729957805909)),
 ('Cluster_1', ('9', 0.1733800350262697, 0.4261603375527425)),
 ('Cluster_3', ('10', 0.5043782837127846, 0.6898734177215189))]

###Finding New Centroids

In [ ]:
# First lambda expression for Within-Partition Reduction Step::
#    x: is a ([v1,v2], v3) that holds: ([runningSum_1, runningSum_2],  runningCount).
#    b: is a SCALAR that holds the next Value

#    Second lambda expression for Cross-Partition Reduction Step::
#    x: is a ([v1,v2], v3) that holds: ([runningSum_1, runningSum_2],  runningCount).
#    b: is a ([v1,v2], v3) that holds: ([nextPartitionsSum_1, nextPartitionsSum_2], nextPartitionsCount)

seqFunc = (lambda x, y  : ([x[0][0] + y[1] , x[0][1] + y[2]], x[1]+1 ))
combFunc = (lambda x, y : ([x[0][0] + y[0][0], x[0][1] + y[0][1]], x[1]+y[1]))

In [ ]:
#Calcualting the first average of clusters.
cus_data_clus_avg = cus_data_clus.aggregateByKey(([0,0], 0), seqFunc, combFunc)\
                                 .mapValues(lambda v: ( v[0][0]/v[1], v[0][1]/v[1])).collect()

In [ ]:
#Printing the first average of the clusters.
cus_data_clus_avg

Out[29]: [('Cluster_3', (0.5628721541155869, 0.7727484359086279)),
 ('Cluster_1', (0.2427853498819767, 0.4068978169143276)),
 ('Cluster_2', (0.9051237969761726, 0.16585375295165108))]

###Updating New Cluster Points

In [ ]:
#The function updates the cluster points after each iteration. 
def update_cluster_points ( clusterPoints ):
    
    Clusters= {
                clusterPoints[0][0]   :  ( clusterPoints[0][1][0], clusterPoints[0][1][1] ),
                clusterPoints[1][0]   :  ( clusterPoints[1][1][0], clusterPoints[1][1][1] ),
                clusterPoints[2][0]   :  ( clusterPoints[2][1][0], clusterPoints[2][1][1] ) 
               }
    
    return (Clusters)

###Delete broadcast variables from cache

In [ ]:
#Delete cached copies of the broadcast on the executors.
def clear_old_cache( ClustersVar ):
    
    ClustersVar.unpersist(True)

In [ ]:
#The function displays the final results as mentioned in the problem statement.
def display_result( consumer_data_rdd_nr_clustering , consumer_data_rdd_nr_clustering_1_avg, iteration ):
    
    if ( iteration == 39) :
        clusterDict = dict()
        consumer_data_rdd_nr_clustering_grp = consumer_data_rdd_nr_clustering.groupByKey().mapValues(list).collect()
        for i in range(3):
            clusterDict[consumer_data_rdd_nr_clustering_grp[i][0]] = list(map ( lambda x : (x[0]), consumer_data_rdd_nr_clustering_grp[i][1] ))[:10]
        
        print ("Final Cluster Points : ", consumer_data_rdd_nr_clustering_1_avg)
        print ("Final Clustered List of Customers : ", clusterDict)
    
    

###K-Means Function for iterations

In [ ]:
#This the main function that calls all the required function to implment K-Means Clustering through Map Reduce for all the iterations.
def K_means_cluster_Normal ( iteration ):
    
    for iteration in range ( iteration ):
       
        if ( iteration == 0 ):  
            consumerData = read_file()
            consumerDataTuple = consumerData.map(lambda x: convert_to_key_value_pair(x))
            minMaxDict = get_min_max_values ( consumerDataTuple )
            consumer_data_rdd_nr = consumerDataTuple.map( lambda x : normalize_value (x, minMaxDict) )
            consumer_data_rdd_nr_list = consumer_data_rdd_nr.collect()
            clusterPoints = set_initial_clusters_points ( consumer_data_rdd_nr_list )
        
        ClustersVar = broad_cast_values ( clusterPoints )
            
        consumer_data_rdd_nr_clustering_1 = consumer_data_rdd_nr.map(lambda line : create_cluster ( line, ClustersVar ) )
        consumer_data_rdd_nr_clustering_1_avg  = consumer_data_rdd_nr_clustering_1.aggregateByKey(([0,0], 0), seqFunc, combFunc)\
                                                                                  .mapValues(lambda v: ( v[0][0]/v[1], v[0][1]/v[1])).collect()

        clear_old_cache( ClustersVar )
        clusterPoints = update_cluster_points( consumer_data_rdd_nr_clustering_1_avg )
        broad_cast_values ( clusterPoints )

    display_result(consumer_data_rdd_nr_clustering_1, consumer_data_rdd_nr_clustering_1_avg, iteration)

###Section -2 : Chain Map Reduce
"Chaining map reduce" to create clusters from the initial data.

In [ ]:
#This the main function that calls all the required function to implment K-Means Clustering through Map Reduce for all the iterations.
def K_means_cluster_chain ( iteration ):
    
    for iteration in range ( iteration ):
       
        if ( iteration == 0 ):  
            consumerDataTuple = read_file().map(lambda x: convert_to_key_value_pair(x))
            minMaxDict = get_min_max_values ( consumerDataTuple )
            consumer_data_rdd_nr = consumerDataTuple.map( lambda x : normalize_value (x, minMaxDict) )
            clusterPoints = set_initial_clusters_points ( consumer_data_rdd_nr.collect() )
        
        ClustersVar = broad_cast_values ( clusterPoints )
            
        consumer_data_rdd_nr_clustering_1_avg = consumer_data_rdd_nr\
                                                                    .map(lambda line : create_cluster ( line, ClustersVar ) )\
                                                                    .aggregateByKey(([0,0], 0), seqFunc, combFunc)\
                                                                    .mapValues(lambda v: ( v[0][0]/v[1], v[0][1]/v[1])).collect()

        clear_old_cache( ClustersVar )
        clusterPoints = update_cluster_points( consumer_data_rdd_nr_clustering_1_avg )
        broad_cast_values ( clusterPoints )


    consumer_data_rdd_nr_clustering_1 = consumer_data_rdd_nr.map(lambda line : create_cluster ( line, ClustersVar ) )
    display_result(consumer_data_rdd_nr_clustering_1, consumer_data_rdd_nr_clustering_1_avg, iteration)

### K-Means Cluster results with normal map reduce method

In [ ]:
K_means_cluster_Normal(40)

Final Cluster Points :  [('Cluster_3', (0.7216941832588033, 0.8408094711400596)), ('Cluster_1', (0.266578800079441, 0.5249032145808866)), ('Cluster_2', (0.9125043782837128, 0.11324894514767939))]
Final Clustered List of Customers :  {'Cluster_3': ['1', '3', '8', '10', '14', '17', '18', '22', '24', '27'], 'Cluster_1': ['4', '5', '6', '9', '16', '19', '21', '25', '26', '28'], 'Cluster_2': ['2', '7', '11', '12', '13', '15', '20', '23', '29', '30']}


### K-Means Cluster results with chaining map reduce method

In [ ]:
K_means_cluster_chain (40)

Final Cluster Points :  [('Cluster_3', (0.7216941832588033, 0.8408094711400596)), ('Cluster_1', (0.266578800079441, 0.5249032145808866)), ('Cluster_2', (0.9125043782837128, 0.11324894514767939))]
Final Clustered List of Customers :  {'Cluster_3': ['1', '3', '8', '10', '14', '17', '18', '22', '24', '27'], 'Cluster_1': ['4', '5', '6', '9', '16', '19', '21', '25', '26', '28'], 'Cluster_2': ['2', '7', '11', '12', '13', '15', '20', '23', '29', '30']}
